## [REINFORCEMENT LEARNING (DQN)](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html#reinforcement-learning-dqn-tutorial)

![](https://pytorch.org/tutorials/_images/cartpole.gif)

#### As the agent observes the current state of the environment and chooses an action, the environment transitions to a new state, and also returns a reward that indicates the consequences of the action. In this task, rewards are +1 for every incremental timestep and the environment terminates if the pole falls over too far or the cart moves more then 2.4 units away from center. This means better performing scenarios will run for longer duration, accumulating larger return.

#### The CartPole task is designed so that the inputs to the agent are 4 real values representing the environment state (position, velocity, etc.). We take these 4 inputs without any scaling and pass them through a small fully-connected network with 2 outputs, one for each action. The network is trained to predict the expected value for each action, given the input state. The action with the highest expected value is then chosen.

In [15]:
import gym

In [16]:
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [18]:
gym.envs.registry.keys()

dict_keys(['CartPole-v0', 'CartPole-v1', 'MountainCar-v0', 'MountainCarContinuous-v0', 'Pendulum-v1', 'Acrobot-v1', 'LunarLander-v2', 'LunarLanderContinuous-v2', 'BipedalWalker-v3', 'BipedalWalkerHardcore-v3', 'CarRacing-v2', 'Blackjack-v1', 'FrozenLake-v1', 'FrozenLake8x8-v1', 'CliffWalking-v0', 'Taxi-v3', 'Reacher-v2', 'Reacher-v4', 'Pusher-v2', 'Pusher-v4', 'InvertedPendulum-v2', 'InvertedPendulum-v4', 'InvertedDoublePendulum-v2', 'InvertedDoublePendulum-v4', 'HalfCheetah-v2', 'HalfCheetah-v3', 'HalfCheetah-v4', 'Hopper-v2', 'Hopper-v3', 'Hopper-v4', 'Swimmer-v2', 'Swimmer-v3', 'Swimmer-v4', 'Walker2d-v2', 'Walker2d-v3', 'Walker2d-v4', 'Ant-v2', 'Ant-v3', 'Ant-v4', 'Humanoid-v2', 'Humanoid-v3', 'Humanoid-v4', 'HumanoidStandup-v2', 'HumanoidStandup-v4'])

In [19]:
if gym.__version__[:4] == '0.26':
    env = gym.make('CartPole-v1')
else:
    raise ImportError(f"Requires gym v26, actual version: {gym.__version__}")

In [20]:
is_ipython = 'inline' in matplotlib.get_backend()

In [21]:
if is_ipython:
    from IPython import display

In [22]:
plt.ion() # interactive mode on

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
device

device(type='cpu')

### [Replay Memory](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html#replay-memory)

#### We’ll be using experience replay memory for training our DQN. It stores the transitions that the agent observes, allowing us to reuse this data later. By sampling from it randomly, the transitions that build up a batch are decorrelated. It has been shown that this greatly stabilizes and improves the DQN training procedure.

In [25]:
Transition = namedtuple('Transition', ('state', 'action', 'nxt_state', 'reward'))

##### ReplayMemory - a cyclic buffer of bounded size that holds the transitions observed recently. It also implements a .sample() method for selecting a random batch of transitions for training.

In [26]:
class ReplayMemory(object):
    def __init__(self, capacity) -> None:
        self.memory = deque([], maxlen = capacity) # buffer store transitions

    def push(self, *args):
        """save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

##### Our environment is deterministic, so all equations presented here are also formulated deterministically for the sake of simplicity. In the RL literature, they would also contain expectations over stochastic transitions in the environment.

##### Our aim will be to train a policy $ \prod $ that tries to maximize the discounted, cumulative reward (DCR) $R_{t_0}= \sum_{t_0}^\infty \gamma^{t-t_0}r_t$ 

#####  A lower $\gamma$ makes rewards from the uncertain far future less important for our agent than the ones in the near future that it can be fairly confident about. It also encourages agents to collect reward closer in time than equivalent rewards temporally future away.

Reward function $Q^*: State \times Action \rightarrow \mathbb{R}$

$\pi^*(s) =\underset{x}{\arg\max} Q^*(s,a)$ take action a in a given state to maximize Q

#### But, since neural networks are universal function approximators, we can simply create one and train it to resemble $Q^*$ 

In [27]:
class DQN(nn.Module):
    def __init__(self, n_obs, n_arms):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_obs, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_arms)
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)


In [28]:
n_arms = env.action_space.n

2

In [30]:
state, _ = env.reset()

In [32]:
n_obs = len(state)

In [33]:
n_obs

4